# Load data

In [2]:
import numpy as np 
from monty.json import MontyDecoder
from monty.serialization import loadfn

data = loadfn('bulk_moduli.json')
structures = data['structures']
targets = np.log10(data['bulk_moduli'])

# Set up model and train

In [3]:
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.models import MEGNetModel

model = MEGNetModel(10, 2, nblocks=1, lr=1e-2,
                    n1=4, n2=4, n3=4, npass=1, ntarget=1,
                    graph_converter=CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 10), 0.5)))

2024-01-22 16:36:24.587381: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 16:36:24.599892: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 16:36:24.599994: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 16:36:24.600529: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
model.train(structures, targets, epochs=2)

Epoch 1/2
1/1 [==============================] - 2s 2s/step - loss: 4.0010
Epoch 2/2
1/1 [==============================] - 0s 61ms/step - loss: 3.8299


2024-01-22 16:36:28.907220: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-22 16:36:28.907251: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


# Prediction

## Predict from structure

In [14]:
from pymatgen.ext.matproj import MPRestError, TaskType, _MPResterLegacy
# rester = _MPResterLegacy(api_key='iAihdZzrZYLQKZms1S43De90NiNK6ABB') # mp-api style
rester = _MPResterLegacy('iTPrDnB1NuSywGXI')  # legacy style get the mp data

/root/anaconda3/envs/megnet/lib/python3.8/site-packages/pymatgen/ext/matproj.py:183: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


In [15]:
rester.api_key

'iTPrDnB1NuSywGXI'

In [16]:
# from pymatgen import MPRester
# mpr = MPRester()

# rester.get_materials_ids("Al2O3")
# let's get some structure
structure = rester.get_structure_by_material_id('mp-1143')

In [17]:
predicted_K = 10 ** model.predict_structure(structure).ravel()
print('The predicted K for {} is {} GPa'.format(structure.formula, predicted_K[0]))

The predicted K for Al4 O6 is 1.1948199272155762 GPa


## matgl 

In [20]:
from pymatgen.core import Lattice, Structure
import matgl

# model = matgl.load_model("MEGNet-MP-2018.6.1-Eform")
model = matgl.load_model("M3GNet-MP-2018.6.1-Eform")

ValueError: No valid model found in pre-trained_models at https://github.com/materialsvirtuallab/matgl/raw/main/pretrained_models/.

In [19]:
import matgl
print(matgl.get_available_pretrained_models())

['M3GNet-MP-2018.6.1-Eform', 'M3GNet-MP-2021.2.8-DIRECT-PES', 'M3GNet-MP-2021.2.8-PES', 'MEGNet-MP-2018.6.1-Eform', 'MEGNet-MP-2019.4.1-BandGap-mfi']
